<div style="direction:rtl">
    <font style="font-family:'Courier New';font-weight:bold;color:Black" size="5">
پروژه CA4 : استفاده از روش HMM برای POS Tagging
    </font>
</div>

In [2]:
import nltk
import xlrd 
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
در بخش اول پس از خواندن فایل viterbi_train.xlsx، متدی به نام sents_tags_tokenize  نوشته شده که اطلاعات دادگان آموزشی را خوانده و لیستی شامل زوج مرتبهای (word,tag) بر می‌گرداند. همچنین در این متد دو تگ _START_ و _END_  به ابتدا و انتهای هر جمله در لیست خروجی اضافه شده است</font>
</div>

In [3]:
train_wb = xlrd.open_workbook('data/viterbi_train.xlsx') 
train_sheet = train_wb.sheet_by_index(0) 

In [4]:
def sents_tags_tokenize(data_sheet, padded, pad_start_symbol, pad_end_symbol):
    word_tags = []    
    curr_s = ''
    for i in range(data_sheet.nrows):
        row = data_sheet.row_values(i)
        s,w,t = row[0], row[1], row[2]
        if (curr_s == ''):
            curr_s = s 
            if padded == True:
                word_tags.append((pad_start_symbol,pad_start_symbol))
                
        elif s != '':
            curr_s = s
            if padded == True:                
                word_tags.append((pad_end_symbol,pad_end_symbol))
                word_tags.append((pad_start_symbol,pad_start_symbol))
        word_tags.append((str(w),t))      
        
    if padded == True:
        word_tags.append((pad_end_symbol,pad_end_symbol))        
    return word_tags

In [5]:
startTag = '_START_'
endTag = '_END_'
word_tags = sents_tags_tokenize(train_sheet, padded=True, pad_start_symbol=startTag, pad_end_symbol=endTag)

In [6]:
print("len of word_tags:",len(word_tags))
print("--------------------------")
print("sample word_tags : ", word_tags[:50])

len of word_tags: 95151
--------------------------
sample word_tags :  [('_START_', '_START_'), ('Palestinian', 'JJ'), ('Authority', 'NNP'), ('President', 'NNP'), ('Mahmoud', 'NNP'), ('Abbas', 'NNP'), ('is', 'VBZ'), ('expected', 'VBN'), ('to', 'TO'), ('ask', 'VB'), ('Hamas', 'NNP'), ('prime', 'JJ'), ('minister-designate', 'JJ'), ('Ismail', 'NNP'), ('Haniyeh', 'NNP'), ('to', 'TO'), ('form', 'VB'), ('a', 'DT'), ('new', 'JJ'), ('government', 'NN'), ('Tuesday', 'NNP'), ('.', '.'), ('_END_', '_END_'), ('_START_', '_START_'), ('Hamas', 'NNP'), ('wants', 'VBZ'), ('to', 'TO'), ('build', 'VB'), ('a', 'DT'), ('coalition', 'NN'), ('government', 'NN'), (',', ','), ('but', 'CC'), ('Mr.', 'NNP'), ('Abbas', 'NNP'), ("'s", 'POS'), ('Fatah', 'NNP'), ('party', 'NN'), ('and', 'CC'), ('the', 'DT'), ('militant', 'JJ'), ('Islamic', 'NNP'), ('Jihad', 'NNP'), ('group', 'NN'), ('have', 'VBP'), ('declined', 'VBN'), ('to', 'TO'), ('join', 'VB'), ('.', '.'), ('_END_', '_END_')]


<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
در بخش بعد برای بدست آوردن ماتریسهای Transition و Emission ابتدا تعداد دفعات تکرار هر pos و تعداد دفعات تکرار ترکیبهای دوتایی posها و همچنین تعداد دفعات تکرار هر زوج word,pos در دادگان آموزشی بدست آمده و سپس ماتریسهای trans_prob_matrix و emission_prob_matrix محاسبه شد. با توجه به اینکه تعداد احتمالات با مقدار صفر در این ماتریسها زیاد بوده لذا از روش add-one برای smoothing استفاده شد. 
</font>
</div>

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
words : مجموعه کلمات یونیک که سایز vocab خواهد بود
</font>
</div>

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
tagCountDict : .تعداد دفعات تکرار هر تگ را نگه مي‌دارد. هر تگی که به عنوان تگ قبلی استفاده شده است
</font>
</div>

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
bigramTagsCountDict : تعداد دفعات تکرار هر دو تگ پشت سر هم را نگه مي‌دارد
</font>
</div>

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
wordTagCountDict : تعداد دفعاتی که به یک کلمه یک tag خاص نسبت داده شده را نگه مي‌دارد
</font>
</div>

In [7]:
words = {startTag,endTag} # set
tagCountDict = {}         # dictionary
bigramTagsCountDict = {}
wordTagCountDict = {}

prevTag = startTag
for word,tag in word_tags:
    words.add(word)
    if prevTag not in tagCountDict:
        tagCountDict[prevTag] = 1
    else:
        tagCountDict[prevTag] = tagCountDict[prevTag] + 1
        
    if (prevTag,tag) not in bigramTagsCountDict:
        bigramTagsCountDict[(prevTag,tag)] = 1
    else:
        bigramTagsCountDict[(prevTag,tag)] = bigramTagsCountDict[(prevTag,tag)] + 1
    prevTag = tag
    
    if (word,tag) not in wordTagCountDict:
        wordTagCountDict[(word,tag)] = 1
    else:
        wordTagCountDict[(word,tag)] = wordTagCountDict[(word,tag)] + 1

In [8]:
tags_size = len(tagCountDict)
vocab_size = len(words)

print("tags_size:", tags_size)
print("vocab_size:", vocab_size)

print("\nlen of tagCountDict:", len(tagCountDict))
print("sample tagCountDict['NN']:", tagCountDict['NN'])

print("\nlen of bigramTagsCountDict:", len(bigramTagsCountDict))
print("sample bigramTagsCountDict[('VB', 'DT')]:", bigramTagsCountDict[('VB', 'DT')])

print("\nlen of wordTagCountDict:", len(wordTagCountDict))
print("sample wordTagCountDict[('hope', 'NN')]:", wordTagCountDict[('hope', 'NN')])

tags_size: 42
vocab_size: 10318

len of tagCountDict: 42
sample tagCountDict['NN']: 12158

len of bigramTagsCountDict: 825
sample bigramTagsCountDict[('VB', 'DT')]: 476

len of wordTagCountDict: 11487
sample wordTagCountDict[('hope', 'NN')]: 9


<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
به هر کلمه و هر تگ، يک ایندکس عددی نسبت داده می شود تا در ماتریس براساس آن ایندکس، ارجاع شوند
</font>
</div>

In [9]:
tags = list(tagCountDict.keys())
tag2index = {t:i for i, t in enumerate(tags)}

In [10]:
words = list(words)
word2index = {w:i for i, w in enumerate(words)}

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
ماتریس Transition که ابعاد آن به اندازه tag_size * tag_size می باشد، ساخته می شود
</font>
</div>

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
هر درایه این ماتریس از تقسیم تعداد دفعات تکرار یک ترکیب دوتایی تگ به دفعات تکرار تگ اولی بدست می آید.
</font>
</div>

In [238]:
# Without Smoothing
# Transition probabilities
trans_prob_matrix = np.zeros([tags_size,tags_size])
for pre_tag,tag in bigramTagsCountDict:
    i = tag2index[pre_tag]
    j = tag2index[tag]
    trans_prob_matrix[i][j] = bigramTagsCountDict[(pre_tag,tag)]/ tagCountDict[pre_tag]

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
 با توجه به اینکه تعداد احتمالات با مقدار صفر در این ماتریس زیاد بوده لذا از روش add-one برای smoothing استفاده شد. فرض شد که یک ترکیب دوتایی برای هر دو تگ انتخابی اضافه شده است (tag_size*tag_size زوج مرتب به دارگان آموزشی اضافه شده)
</font>
</div>

In [11]:
# With Smoothing add-one
# Transition probabilities
trans_prob_matrix = np.zeros([tags_size,tags_size])
for t_i in tags:
    for t_j in tags:
        i = tag2index[t_i]
        j = tag2index[t_j]        
        if (t_i,t_j) in bigramTagsCountDict:
            trans_prob_matrix[i][j] = (bigramTagsCountDict[(t_i,t_j)]+1) / (tagCountDict[t_i]+tags_size)
        else:
            trans_prob_matrix[i][j] = 1/(tagCountDict[t_i]+tags_size)

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
برای کنترل صحت مقداردهی ماتریس Transition، به ازای هر تگ، تمام احتمالات تگهای بعدی آن با هم جمع شد که باید برابر 1 شود.
</font>
</div>

In [12]:
# Test of correctness of Transition Matrix
for i in range(len(tags)):
    sum = 0
    for j in range(len(tags)):
        sum = sum + trans_prob_matrix[i][j] 
    print ("sum trans prob of tag ",tags[i],"=",sum)

sum trans prob of tag  _START_ = 0.999999999999999
sum trans prob of tag  JJ = 1.0
sum trans prob of tag  NNP = 1.0000000000000004
sum trans prob of tag  VBZ = 0.9999999999999996
sum trans prob of tag  VBN = 0.9999999999999996
sum trans prob of tag  TO = 1.0000000000000007
sum trans prob of tag  VB = 1.0
sum trans prob of tag  DT = 1.0000000000000004
sum trans prob of tag  NN = 0.9999999999999997
sum trans prob of tag  . = 0.9999999999999984
sum trans prob of tag  _END_ = 0.9999999999999988
sum trans prob of tag  , = 1.0
sum trans prob of tag  CC = 0.9999999999999999
sum trans prob of tag  POS = 0.9999999999999993
sum trans prob of tag  VBP = 0.9999999999999997
sum trans prob of tag  VBG = 1.0000000000000002
sum trans prob of tag  IN = 1.0000000000000007
sum trans prob of tag  NNS = 1.0000000000000002
sum trans prob of tag  PRP = 1.0000000000000007
sum trans prob of tag  RP = 1.0
sum trans prob of tag  VBD = 1.0000000000000002
sum trans prob of tag  MD = 1.0000000000000007
sum trans pr

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
ماتریس Emission که ابعاد آن به اندازه vocab_size * tag_size می باشد، ساخته می شود
</font>
</div>

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
هر درایه این ماتریس از تقسیم تعداد دفعات تخصیص یک تگ به یک کلمه به دفعات تکرار آن تگ بدست می آید.
</font>
</div>

In [245]:
# Without Smoothing
# Emission probabilities
emission_prob_matrix = np.zeros([vocab_size,tags_size])
for word,tag in wordTagCountDict:
    i = word2index[word]
    j = tag2index[tag]
    emission_prob_matrix[i][j] = wordTagCountDict[(word,tag)]/tagCountDict[tag]

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
 با توجه به اینکه تعداد احتمالات با مقدار صفر در این ماتریس زیاد بوده لذا از روش add-one برای smoothing استفاده شد. فرض شد که یک ترکیب دوتایی برای هر کلمه و تگ   اضافه شده است (word_size*tag_size زوج مرتب به دارگان آموزشی اضافه شده)
</font>
</div>

In [13]:
# With Smoothing add-one
# Emission probabilities
emission_prob_matrix = np.zeros([vocab_size,tags_size])
for w_i in words:
    for t_j in tags:
        i = word2index[w_i]
        j = tag2index[t_j]        
        if (w_i,t_j) in wordTagCountDict:
            emission_prob_matrix[i][j] = (wordTagCountDict[(w_i,t_j)]+1)/ (tagCountDict[t_j]+vocab_size)
        else:
            emission_prob_matrix[i][j] = 1/(tagCountDict[t_j]+vocab_size)

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
برای کنترل صحت مقداردهی ماتریسEmission، به ازای هر تگ، تمام احتمالات تخصیص آن به کلمات مختلف با هم جمع شد که باید برابر 1 شود.
</font>
</div>

In [14]:
# Test of correctness of Emission Matrix
for i in range(len(tags)):
    sum = 0
    for j in range(len(words)):
        sum = sum + emission_prob_matrix[j][i] 
    print ("sum emission prob of tag ",tags[i],"=",sum)

sum emission prob of tag  _START_ = 0.9999301627207909
sum emission prob of tag  JJ = 1.0000000000001843
sum emission prob of tag  NNP = 0.9999999999999767
sum emission prob of tag  VBZ = 1.0000000000001812
sum emission prob of tag  VBN = 1.0000000000001892
sum emission prob of tag  TO = 1.0000000000000877
sum emission prob of tag  VB = 1.0000000000002072
sum emission prob of tag  DT = 1.0000000000001723
sum emission prob of tag  NN = 1.0000000000001719
sum emission prob of tag  . = 1.0000000000002072
sum emission prob of tag  _END_ = 1.0000698470349012
sum emission prob of tag  , = 0.9999999999998457
sum emission prob of tag  CC = 1.0000000000000442
sum emission prob of tag  POS = 1.000000000000121
sum emission prob of tag  VBP = 0.9999999999999067
sum emission prob of tag  VBG = 1.0000000000001656
sum emission prob of tag  IN = 1.0000000000002238
sum emission prob of tag  NNS = 0.9999999999999005
sum emission prob of tag  PRP = 1.0000000000001208
sum emission prob of tag  RP = 0.9999

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
الگوریتم viterbi پیاده سازی شده است. ورودی آن یک sequence از کلمات یک جمله است و خروجی آن sequence ای از تگ های تخصیص داده شده به کلمات جمله ورودی می باشد
</font>
</div>

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
در این الگوریتم با استفاده از روش dynamic programming ماتریس viterbi و backtrack به ابعاد تعداد تگها در تعداد کلمات ورودی ساخته و پر می شود.
</font>
</div>

In [15]:
def viterbi(Seq):    
    seq_len = len(Seq)
    h, w = tags_size, seq_len
    viterbi_matrix = [[0 for x in range(w)] for y in range(h)]
    backtrack_matrix = [[0 for x in range(w)] for y in range(h)]
    #initialize first column of viterbi_matrix
    for s in tags:
        state_ind = tag2index[s]
        curr_word = Seq[0]
        
        if curr_word in words:
            curr_word_ind = word2index[curr_word]
            multPE = emission_prob_matrix[curr_word_ind][state_ind]
        else:
            multPE = 1
        
        multPT = trans_prob_matrix[tag2index[startTag]][state_ind]
        
        viterbi_matrix[state_ind][0] = multPT * multPE
        backtrack_matrix[state_ind][0] = 0
        
    score = 0       
    for t in range(1, seq_len):
        for s_to in tags: 
            s_to_ind = tag2index[s_to]
            if Seq[t] in words:
                curr_word_ind = word2index[Seq[t]]
                multPE = emission_prob_matrix[curr_word_ind][s_to_ind]
            else:
                multPE = 1
            # find best score from previous step
            f = 0
            for s_from in tags: 
                s_from_ind = tag2index[s_from]
                multPT = trans_prob_matrix[s_from_ind][s_to_ind]
                score = viterbi_matrix[s_from_ind][t-1] * multPT * multPE
                
                if f == 0:
                    viterbi_matrix[s_to_ind][t] = score
                    backtrack_matrix[s_to_ind][t] = s_from
                    f = 1
                if score > viterbi_matrix[s_to_ind][t]:
                    viterbi_matrix[s_to_ind][t] = score
                    backtrack_matrix[s_to_ind][t] = s_from
                else:
                    continue
                    
    result_pairs = []
    best_score_ind = 0
    for i in range(len(tags)):
        if viterbi_matrix[i][seq_len-1] > viterbi_matrix[best_score_ind][seq_len-1]:
            best_score_ind = i
    
    result_pairs.append((Seq[seq_len-1],tags[best_score_ind]))    
    for t in range(seq_len-1, 0, -1):
        #print(t)
        best_tag = backtrack_matrix[best_score_ind][t]
        if best_tag != 0:
            best_score_ind = tag2index[best_tag]
        else:
            best_score_ind=0
        result_pairs.insert(0,(Seq[t-1],best_tag))
    return result_pairs

In [16]:
viterbi(['U.S.-led', 'coalition', 'forces', 'prevented', 'another', 'attack', 'by', 'discovering', 
         'a', 'bomb', 'near', 'the', 'eastern', 'city', 'of', 'Jalalabad', '.'])

[('U.S.-led', 'JJ'),
 ('coalition', 'NN'),
 ('forces', 'NNS'),
 ('prevented', 'IN'),
 ('another', 'DT'),
 ('attack', 'NN'),
 ('by', 'IN'),
 ('discovering', 'VBG'),
 ('a', 'DT'),
 ('bomb', 'NN'),
 ('near', 'IN'),
 ('the', 'DT'),
 ('eastern', 'JJ'),
 ('city', 'NN'),
 ('of', 'IN'),
 ('Jalalabad', 'NNP'),
 ('.', '.')]

<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
حال دادگان تست به کمک متد new_sents_tags_tokenize  به صورت دو لیست از sequence ها تبدیل شد. یک لیست برای sequence های جملات و یک لیست حاوی sequence های تگهای صحیح متناظر.  سپس sequence کلمات هر جمله به الگوریتم viterbi داده شد و تگهای predict شده بدست آمد و سپس به کمک کتابخانه sklearn.metrics مقادیر accuracy، recall و precision محاسبه گردید
</font>
</div>

In [17]:
test_wb = xlrd.open_workbook('data/viterbi_test.xlsx') 
test_sheet = test_wb.sheet_by_index(0) 

In [18]:
def new_sents_tags_tokenize(data_sheet, padded, pad_start_symbol, pad_end_symbol):
    sents = []
    sents_tags = []
    
    curr_s = ''
    curr_sent=[]
    curr_sent_tags=[]
    for i in range(data_sheet.nrows):
        row = data_sheet.row_values(i)
        s,w,t = row[0], row[1], row[2]
        if (curr_s == ''):
            curr_s = s 
            if padded == True:
                curr_sent.append(pad_start_symbol)
                curr_sent_tags.append(pad_start_symbol)
                
        elif s != '':
            curr_s = s
            if padded == True:
                curr_sent.append(pad_end_symbol)
                curr_sent_tags.append(pad_end_symbol)
            sents.append(curr_sent)
            sents_tags.append(curr_sent_tags)
            
            curr_sent=[]
            curr_sent_tags=[]
            if padded == True:
                curr_sent.append(pad_start_symbol)
                curr_sent_tags.append(pad_start_symbol)

        curr_sent.append(str(w))
        curr_sent_tags.append(t)
        
    if  len(curr_sent) > 0:
        if padded == True:
            curr_sent.append(pad_end_symbol)
            curr_sent_tags.append(pad_end_symbol)
        
        sents.append(curr_sent)
        sents_tags.append(curr_sent_tags)
        
    return sents, sents_tags

In [19]:
sents, sents_tags = new_sents_tags_tokenize(test_sheet, padded=True, pad_start_symbol=startTag, pad_end_symbol=endTag)

In [20]:
print("len of sents:",len(sents),"\nlen of sents_tags:",len(sents_tags))
print("--------------------------")
print("sample sentence : ", sents[1])
print("sample sentence tags : ", sents_tags[1])

len of sents: 1001 
len of sents_tags: 1001
--------------------------
sample sentence :  ['_START_', 'The', 'press', 'freedom', 'group', 'Reporters', 'Without', 'Borders', 'has', 'condemned', 'the', 'decision', 'by', 'Venezuelan', 'President', 'Hugh', 'Chavez', 'to', 'shut', 'down', 'one', 'of', 'the', 'country', "'s", 'oldest', 'television', 'stations', 'The', 'group', 'issued', 'a', 'statement', 'Friday', 'calling', 'the', 'move', '"', 'a', 'serious', 'attack', 'against', 'editorial', 'pluralism', '.', '"', '_END_']
sample sentence tags :  ['_START_', 'DT', 'NN', 'NN', 'NN', 'VBZ', 'IN', 'NNS', 'VBZ', 'VBN', 'DT', 'NN', 'IN', 'JJ', 'NNP', 'NNP', 'NNP', 'TO', 'VB', 'RP', 'CD', 'IN', 'DT', 'NN', 'POS', 'JJS', 'NN', 'NNS', 'DT', 'NN', 'VBD', 'DT', 'NN', 'NNP', 'VBG', 'DT', 'NN', '``', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NN', '.', '``', '_END_']


In [21]:
test_tags = []
for s in sents_tags:
    for t in s:
        test_tags.append(t)
test_tags = set(test_tags)        
print("size of different tags in test data:", len(test_tags))
print("tags of test data that aren't in train data", test_tags - set(tags))
print("tags of train data that aren't in test data", set(tags) - test_tags)

size of different tags in test data: 42
tags of test data that aren't in train data {'UH'}
tags of train data that aren't in test data {'PDT'}


<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
با توجه به اینکه در داده های تست تگ UH وجود دارد که در داده های تست نبوده لذا باید هنگام ارزیابی آنرا در نظر گرفت و شرطهای لازم را برای جلوگیری از خطا اعمال کرد
</font>
</div>

In [31]:
y_test = []
y_pred = []
#sents = sents[:200]
for i in range(len(sents)):
    assigned_tags = viterbi(sents[i])
    
    for j in range(len(sents[i])):
        if sents_tags[i][j] in tags:
            y_test.append(tag2index[sents_tags[i][j]])
        else:
            y_test.append(50) # for tag UH that not exists in tags of train data
        y_pred.append(tag2index[assigned_tags[j][1]])


In [33]:
print("precision : ", precision_score(y_test, y_pred, average="micro"))
print("recall : ", recall_score(y_test, y_pred, average="micro")) 
print("f1_score : ", f1_score(y_test, y_pred, average="micro"))
print("accuracy_score : ", accuracy_score(y_test, y_pred))
#print(confusion_matrix(y_test, y_pred))

precision :  0.880666584168626
recall :  0.880666584168626
f1_score :  0.880666584168626
accuracy_score :  0.880666584168626


<div style="direction:rtl">
<font style="font-family:verdana" size="3" color="Black">    
نتایج بدست آمده در یک فایل csv نوشته شد تا بعدا به ستون F فایل viterbi_test.xlsx اضافه شود.
</font>
</div>

In [37]:
start_tag_ind = tag2index[startTag]
end_tag_ind = tag2index[endTag]
result_tags = []
for i in range(len(y_test)):    
    if y_test[i] != start_tag_ind and y_test[i] != end_tag_ind:
        result_tags.append(tags[y_pred[i]])
len(result_tags)   

22241

In [36]:
test_hmm_file = open('test_hmm_tags.csv', "w")    
np.savetxt(test_hmm_file, result_tags, fmt="%s", delimiter=",")
test_hmm_file.close()